- torch molecule つかう

In [1]:
import os

is_kaggle_notebook = os.path.exists("/kaggle/input")

# 必要パッケージをインストール
if is_kaggle_notebook:
    !pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl
    !pip install /kaggle/input/torch-geometric-2-6-1/torch_geometric-2.6.1-py3-none-any.whl

In [2]:
import os
import sys
import warnings
from pathlib import Path
from glob import glob

import pandas as pd

if is_kaggle_notebook:
    sys.path.append("/kaggle/input/torch-molecule-src/torch-molecule")

warnings.filterwarnings("ignore")

In [3]:
pr_number = 1

In [4]:
if is_kaggle_notebook:
    module_path = f"/kaggle/input/myproject-pr-{pr_number:04}"
    !mkdir src
    !cp -r $module_path/* src/
    src_path = "./"
else:
    src_path = "../"

sys.path.append(src_path)

from src.data import add_descriptors, add_external_data, load_data
from src.model import get_model
from src.utils import NULL_FOR_SUBMISSION, generate_scaffold, score

In [5]:
exp = "exp018"
model_name = "gnn"
dataset_id = f"model-{exp}"
targets = ["Tg", "FFV", "Tc", "Density", "Rg"]

if is_kaggle_notebook:
    model_paths = list(glob(f"/kaggle/input/{dataset_id}/*.pt"))
    data_dir = Path("/kaggle/input")
else:
    model_paths = list(glob(f"../outputs/{exp}/model/*.pt"))
    data_dir = Path("../data/raw")

print(model_paths)

_, test = load_data(data_dir)

submission = pd.read_csv(
    data_dir / "neurips-open-polymer-prediction-2025/sample_submission.csv"
)

X_test = test["SMILES"].to_list()

use_model_paths = [path for path in model_paths if model_name in path]

for model_path in use_model_paths:
    model = get_model(model_name)()
    
    model.load(model_path)
    sub = model.predict(X_test)["prediction"]

    for idx, target in enumerate(targets):
        submission[target] += sub[:, idx] / len(model_paths)
    

['../outputs/exp018/model/grea_3.pt', '../outputs/exp018/model/gnn_2.pt', '../outputs/exp018/model/grea_4.pt', '../outputs/exp018/model/grea_2.pt', '../outputs/exp018/model/gnn_4.pt', '../outputs/exp018/model/gnn_1.pt', '../outputs/exp018/model/grea_1.pt', '../outputs/exp018/model/grea_0.pt', '../outputs/exp018/model/gnn_0.pt', '../outputs/exp018/model/gnn_3.pt']


ValueError: Error loading model from ../outputs/exp018/model/gnn_2.pt: No module named 'torch_molecule'

In [7]:
if is_kaggle_notebook:
    submission.to_csv("submission.csv", index=False)
    print("✅ submission saved to submission.csv")